# Scraping by Fight / Fighter

I believe this is necessary, but not 100% sure, due to BFO's grouping of events.

In [1]:

# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil      
import datetime
from scipy.stats import norm
from random import randint
import  random
import os
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
path = os.getcwd()

We will have to one-by-one search and scrape. 

In [ ]:
example_name = 'Dustin Poirier'
example_name_form = example_name.lower().replace(' ','+')
example_name_final = example_name.replace(' ','-')

In [ ]:
def get_fighter_bfo_link(fighter_name):
    example_name_form = fighter_name.lower().replace(' ','+')
    example_name_final = fighter_name.replace(' ','-')
    example_url = 'https://www.bestfightodds.com/search?query=' + example_name_form
    driver.get(example_url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # get table
    table = soup.find('table')
    # read table text and links
    table_text = []
    table_links = []
    for row in table.find_all('tr'):
        for cell in row.find_all('td'):
            table_text.append(cell.text)
            for link in cell.find_all('a'):
                table_links.append(link.get('href'))

    # create dataframe
    table_text = np.array(table_text)
    # as list
    table_text = table_text.tolist()
    # drop empty elements
    table_text = [x for x in table_text if x != '']
    table_links = np.array(table_links)
    # as list
    table_links = table_links.tolist()
    # make df for table links and text
    df_table_text = pd.DataFrame(table_text)
    df_table_links = pd.DataFrame(table_links)
    # merge
    df_table = pd.concat([df_table_text, df_table_links], axis = 1)
    # rename cols
    df_table.columns = ['Name','link']
    try:
        # find the fighter link
        fighter_link = df_table[df_table['Name'] == fighter_name]['link'].values[0]
        # get fighter link
        fighter_link = 'https://www.bestfightodds.com' + fighter_link
        # wait 2 seconds
        time.sleep(1)
    except:
        fighter_link = 'NA'
    return fighter_link

Get all UFC Fighters

In [ ]:
df = pd.read_csv('data/final/aggregates/Double_Fights_DF_V2.csv')
all_fighters = df['Fighter_A'].unique().tolist()
all_fighters = pd.DataFrame(all_fighters)
all_fighters.columns = ['Name']
all_fighters

In [ ]:
# test
fighter_name = 'Alex Caceres'
get_fighter_bfo_link(fighter_name)

In [ ]:
# # take list of fighters and get links, save links as you go
# fighter_links = []
# for fighter in all_fighters['Name']:
#     fighter_links.append(get_fighter_bfo_link(fighter))

# # save links
# fighter_links = pd.DataFrame(fighter_links)
# fighter_links.columns = ['link']
# fighter_links.to_csv('data/final/aggregates/Fighter_Links.csv', index = False)
# fighter_links

The above should go for about 100 minutes to get them all

In [ ]:
# # add links to all_fighters
# all_fighters = pd.concat([all_fighters, fighter_links], axis = 1)
# all_fighters

In [ ]:
# all_fighters.to_csv('data/final/aggregates/BFO_All_Fighters_Urls.csv', index = False)

In [ ]:
# load
all_fighters = pd.read_csv('data/final/aggregates/BFO_All_Fighters_Urls.csv')

In [ ]:
all_fighters.head(3)

In [ ]:
# get link 1
fighter_links = all_fighters['link'].tolist()
# drop nan fighter_links
fighter_links = [x for x in fighter_links if str(x) != 'nan']
fighter_links

In [ ]:
driver = webdriver.Chrome(path + '/chromedriver')

### Run BFO Player Page Scrape from Here

In [1]:

# Load Packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import requests     
import shutil      
import datetime
from scipy.stats import norm
from random import randint
import  random
import os
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
path = os.getcwd()

In [2]:
def get_fighter_bfo_page(fighter_link, fighter_name):
    driver.get(fighter_link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # get table
    table = soup.find('table')
    if table is None:
        print(f"No table found on page for fighter {fighter_name}")    
    # read with read_html
    df_table = pd.read_html(str(table))[0]
    try:
        # drop every third row, starting with 0
        df_table = df_table.drop(df_table.index[::3])
        # reset index
        df_table = df_table.reset_index(drop = True)
        # drop Movement col
        df_table = df_table.drop(['Movement', 'Closing range.1'], axis = 1)
        # make player table and opponent table
        df_table_player = df_table.iloc[::2]
        df_table_opponent = df_table.iloc[1::2]
        # reset index
        df_table_player = df_table_player.reset_index(drop = True)
        df_table_opponent = df_table_opponent.reset_index(drop = True)
        # rename opponent cols
        df_table_opponent.columns = ['Opponent', 'Opp_Open', 'Opp_Closing_Range', 'Opp_Closing_Range_2', 'Opp_Odds_Change', 'Date']
        # merge
        df_table_player = pd.concat([df_table_player, df_table_opponent], axis = 1)
        # save to csv
        df_table_player.to_csv('data/final/odds/fighter_BFO_pages/' + fighter_name + '.csv', index = False)
        return df_table_player
    
    except:
        print('Error with ' + fighter_name)
        return 'NA'


In [3]:
all_fighters = pd.read_csv('data/final/aggregates/BFO_All_Fighters_Urls.csv')
# check for already downloaded pages
downloaded_pages = os.listdir('data/final/odds/fighter_BFO_pages/')
downloaded_pages = [x.replace('.csv','') for x in downloaded_pages]

In [4]:
#filter out fighters with pages already downloaded from all_fighters
all_fighters = all_fighters[~all_fighters['Name'].isin(downloaded_pages)]
len(all_fighters)
fighter_links = all_fighters['link'].tolist()
# drop na
fighter_links = [x for x in fighter_links if str(x) != 'nan']
# drop empty strings
fighter_links = [x for x in fighter_links if str(x) != '']
len(fighter_links)

961

In [5]:
driver = webdriver.Chrome(path + '/chromedriver')

In [6]:
for link in fighter_links:
    try:
        fighter_name = all_fighters[all_fighters['link'] == link]['Name'].values[0]
        get_fighter_bfo_page(link, fighter_name)
        print(fighter_name)
        time.sleep(2)
    except:
        print(link + 'NA')
        time.sleep(1)
        continue


Christian Morecraft
Yair Rodriguez
Dan Cramer
Niklas Stolze
Vagner Rocha
Mike Rodriguez
Parker Porter
Ramsey Nijem
Matt Frevola
Trey Ogden
Sodiq Yusuff
Kenny Robertson
Frank Camacho
Matt Dwyer
Eric Schafer
Miguel Baeza
Maciej Jewtuszko
Jim Alers
George Roop
Allan Nascimento
Gregor Gillespie
Cat Zingano
Caio Magalhaes
Youssef Zalal
Mana Martinez
Nolan Ticman
Heath Herring
Francimar Barroso
Jordan Rinaldi
Muslim Salikhov
Mike Russow
Gavin Tucker
Roman Kopylov
Alan Jouban
Dustin Ortiz
Roan Carneiro
Devonte Smith
Raphael Assuncao
Nassourdine Imavov
Dongi Yang
Chris Gruetzemacher
Ryan Benoit
Marcelo Guimaraes
Chris Clements
Gillian Robertson
Vaughan Lee
Dwight Grant
Jordan Williams
Augusto Montano
Artem Lobov
Tomasz Drwal
Mackenzie Dern
Michael Morales
Danny Castillo
Quinton Jackson
Martin Bravo
Hector Urbina
Wellington Turman
Kyle Noke
Gregory Rodrigues
William Macario
Hector Lombard
Mickey Gall
Corey Anderson
Muhammad Mokaev
Luke Sanders
Uros Medic
Ron Stallings
Joel Alvarez
Marcos Vinici

In [7]:
# open a random one
random_page = random.choice(os.listdir('data/final/odds/fighter_BFO_pages/'))
# open csv
df = pd.read_csv('data/final/odds/fighter_BFO_pages/' + random_page)
df.head(3)

,Matchup,Open,Closing range,Closing range.2,Unnamed: 6,Event,Opponent,Opp_Open,Opp_Closing_Range,Opp_Closing_Range_2,Opp_Odds_Change,Date
0,Roger Zapata,-110,-105,-105,+2.2%▲,UFC Fight Night 75: Nelson vs. Barnett,Shinsho Anzai,-130,-115,-115,+2.2%▲,Sep 26th 2015


In [2]:
# get all fighter pages, append to one df
all_fighter_pages = os.listdir('data/final/odds/fighter_BFO_pages/')

for page in all_fighter_pages:
    df = pd.read_csv('data/final/odds/fighter_BFO_pages/' + page)
    try:
        df_final = pd.concat([df_final, df], axis = 0)
    except:
        df_final = df

df_final

,Matchup,Open,Closing range,Closing range.2,Unnamed: 6,Event,Opponent,Opp_Open,Opp_Closing_Range,Opp_Closing_Range_2,Opp_Odds_Change,Date
0,Corey Hill,-210.0,-160.0,-160.0,+9.2%▲,XFC 21: Night Of Champions,Ryan Thomas,160.0,140.0,140.0,+9.2%▲,Dec 7th 2012
1,Corey Hill,-315.0,-315.0,-315.0,NaN,UFC Fight Night 16: Fight For The Troops,Dale Hartt,245.0,245.0,245.0,NaN,Dec 10th 2008
2,Corey Hill,-240.0,-235.0,-235.0,+0.6%▲,UFC 86: Jackson vs Griffin,Justin Buchholz,200.0,195.0,195.0,+0.6%▲,Jul 5th 2008
3,Corey Hill,-195.0,-160.0,-160.0,+6.9%▲,UFC Fight Night 12: Swick vs Burkman,Joe Veres,160.0,140.0,140.0,+6.9%▲,Jan 23rd 2008
4,Corey Hill,NaN,NaN,NaN,NaN,XFC 22: Crossing The Line,Roger Carroll,NaN,NaN,NaN,NaN,Feb 22nd 2013
...,...,...,...,...,...,...,...,...,...,...,...,...
4,Marcus Aurelio,-800.0,-800.0,-800.0,0%,UFC Fight Night 13: Florian vs Lauzon,Ryan Roberts,500.0,550.0,550.0,0%,Apr 2nd 2008
5,Marcus Aurelio,-495.0,-450.0,-450.0,+1.7%▲,UFC 78: Validation,Luke Caudillo,360.0,360.0,360.0,+1.7%▲,Nov 17th 2007
6,Marcus Aurelio,-120.0,105.0,105.0,+10.6%▲,UFC 74: Respect,Clay Guida,-120.0,-125.0,-125.0,+10.6%▲,Aug 25th 2007
7,Marcus Aurelio,NaN,NaN,NaN,NaN,Bellator Fighting Championship 79,Christ Franck,NaN,NaN,NaN,NaN,Nov 2nd 2012


In [3]:
# rename matchup column to fighter
df_final = df_final.rename(columns = {'Matchup':'Fighter'})
# drop Closing range.2
df_final = df_final.drop('Closing range.2', axis = 1)
# Drop Unnamed: 6
df_final = df_final.drop('Unnamed: 6', axis = 1)
# Drop Opp_Closing_Range_2
df_final = df_final.drop('Opp_Closing_Range_2', axis = 1)
# Drop Opp_Odds_Change
df_final = df_final.drop('Opp_Odds_Change', axis = 1)
df_final

,Fighter,Open,Closing range,Event,Opponent,Opp_Open,Opp_Closing_Range,Date
0,Corey Hill,-210.0,-160.0,XFC 21: Night Of Champions,Ryan Thomas,160.0,140.0,Dec 7th 2012
1,Corey Hill,-315.0,-315.0,UFC Fight Night 16: Fight For The Troops,Dale Hartt,245.0,245.0,Dec 10th 2008
2,Corey Hill,-240.0,-235.0,UFC 86: Jackson vs Griffin,Justin Buchholz,200.0,195.0,Jul 5th 2008
3,Corey Hill,-195.0,-160.0,UFC Fight Night 12: Swick vs Burkman,Joe Veres,160.0,140.0,Jan 23rd 2008
4,Corey Hill,NaN,NaN,XFC 22: Crossing The Line,Roger Carroll,NaN,NaN,Feb 22nd 2013
...,...,...,...,...,...,...,...,...
4,Marcus Aurelio,-800.0,-800.0,UFC Fight Night 13: Florian vs Lauzon,Ryan Roberts,500.0,550.0,Apr 2nd 2008
5,Marcus Aurelio,-495.0,-450.0,UFC 78: Validation,Luke Caudillo,360.0,360.0,Nov 17th 2007
6,Marcus Aurelio,-120.0,105.0,UFC 74: Respect,Clay Guida,-120.0,-125.0,Aug 25th 2007
7,Marcus Aurelio,NaN,NaN,Bellator Fighting Championship 79,Christ Franck,NaN,NaN,Nov 2nd 2012


In [4]:
# add implied probabilities
def american_odds_to_implied_probability(american_odds):
    """
    Convert American betting odds to implied probability.

    Parameters:
    american_odds (int): American odds, positive for underdog, negative for favorite

    Returns:
    float: The implied probability as a percentage
    """
    
    if american_odds > 0:
        # For positive odds: Implied Probability = 100 / (American Odds + 100)
        implied_probability = 100 / (american_odds + 100)
    elif american_odds < 0:
        # For negative odds: Implied Probability = -American Odds / (-American Odds + 100)
        implied_probability = -american_odds / (-american_odds + 100)
    else:
        return np.nan

    return implied_probability * 100  # Converting decimal to percentage


In [5]:
# add implied probability to Open, Closing Range, Opp_Open, Opp_Closing_Range
df_final['Open_Implied_Prob'] = df_final['Open'].apply(american_odds_to_implied_probability).round(2)
df_final['Closing_Range_Implied_Prob'] = df_final['Closing range'].apply(american_odds_to_implied_probability).round(2)
df_final['Opp_Open_Implied_Prob'] = df_final['Opp_Open'].apply(american_odds_to_implied_probability).round(2)
df_final['Opp_Closing_Range_Implied_Prob'] = df_final['Opp_Closing_Range'].apply(american_odds_to_implied_probability).round(2)
# add change in implied probability
df_final['Change_in_Implied_Prob'] = df_final['Closing_Range_Implied_Prob'] - df_final['Open_Implied_Prob']
df_final['Opp_Change_in_Implied_Prob'] = df_final['Opp_Closing_Range_Implied_Prob'] - df_final['Opp_Open_Implied_Prob']
df_final

,Fighter,Open,Closing range,Event,Opponent,Opp_Open,Opp_Closing_Range,Date,Open_Implied_Prob,Closing_Range_Implied_Prob,Opp_Open_Implied_Prob,Opp_Closing_Range_Implied_Prob,Change_in_Implied_Prob,Opp_Change_in_Implied_Prob
0,Corey Hill,-210.0,-160.0,XFC 21: Night Of Champions,Ryan Thomas,160.0,140.0,Dec 7th 2012,67.74,61.54,38.46,41.67,-6.20,3.21
1,Corey Hill,-315.0,-315.0,UFC Fight Night 16: Fight For The Troops,Dale Hartt,245.0,245.0,Dec 10th 2008,75.90,75.90,28.99,28.99,0.00,0.00
2,Corey Hill,-240.0,-235.0,UFC 86: Jackson vs Griffin,Justin Buchholz,200.0,195.0,Jul 5th 2008,70.59,70.15,33.33,33.90,-0.44,0.57
3,Corey Hill,-195.0,-160.0,UFC Fight Night 12: Swick vs Burkman,Joe Veres,160.0,140.0,Jan 23rd 2008,66.10,61.54,38.46,41.67,-4.56,3.21
4,Corey Hill,NaN,NaN,XFC 22: Crossing The Line,Roger Carroll,NaN,NaN,Feb 22nd 2013,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Marcus Aurelio,-800.0,-800.0,UFC Fight Night 13: Florian vs Lauzon,Ryan Roberts,500.0,550.0,Apr 2nd 2008,88.89,88.89,16.67,15.38,0.00,-1.29
5,Marcus Aurelio,-495.0,-450.0,UFC 78: Validation,Luke Caudillo,360.0,360.0,Nov 17th 2007,83.19,81.82,21.74,21.74,-1.37,0.00
6,Marcus Aurelio,-120.0,105.0,UFC 74: Respect,Clay Guida,-120.0,-125.0,Aug 25th 2007,54.55,48.78,54.55,55.56,-5.77,1.01
7,Marcus Aurelio,NaN,NaN,Bellator Fighting Championship 79,Christ Franck,NaN,NaN,Nov 2nd 2012,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Save to csv
df_final.to_csv('data/final/odds/BestFightOdds_All_Fighter_Odds_by_Fight.csv', index = False)
df_final.to_csv('data/a_minimal/Odds_By_Fighter.csv', index = False)